# Additional Material

## Overview

This notebook covers different bits and pieces of OpenMP that are relevant in specialized cases.

In [ ]:
%load_ext ice.magic

## OpenMP Version

When OpenMP is enabled, the `_OPENMP` macro name is defined.
It holds information about the OpenMP version supported and is encoded as date in the form of `yyyymm`.

In [ ]:
%%cpp_omp -o code/additional/omp-version.cpp

std::cout << _OPENMP << std::endl;


| Value  | Date     | Version |
|--------|----------|---------|
| 199810 | Oct 1998 | 1.0     |
| 200203 | Mar 2002 | 2.0     |
| 200505 | May 2005 | 2.5     |
| 200805 | May 2008 | 3.0     |
| 201107 | Jul 2011 | 3.1     |
| 201307 | Jul 2013 | 4.0     |
| 201511 | Nov 2015 | 4.5     |
| 201811 | Nov 2018 | 5.0     |
| 202011 | Nov 2020 | 5.1     |
| 202111 | Nov 2021 | 5.2     |

## Directive Formats


OpenMP support different directive formats ([OpenMP 5.1 - 2.1](https://www.openmp.org/spec-html/5.1/openmpse9.html)).

In [ ]:
%%cpp_omp -o code/additional/directives.cpp

//# pragma directive
#pragma omp parallel num_threads(4)
    #pragma omp critical
        std::cout << omp_get_thread_num() << std::endl;

//# pragma operator
_Pragma("omp parallel num_threads(4)")
    _Pragma("omp critical")
        std::cout << omp_get_thread_num() << std::endl;

//# c++ attribute specifier
[[ omp :: sequence(directive(parallel, num_threads(4)), directive(critical)) ]]
        std::cout << omp_get_thread_num() << std::endl;

## Sections

OpenMP supports 'unstructured' parallelism via parallel sections ([OpenMP 5.1 - 2.10.1](https://www.openmp.org/spec-html/5.1/openmpsu42.html)).
\
Inside a parallel region, this is done by spanning a `sections` region.
Inside, multiple `section` items are specified, and each is executed by one of the threads of the enclosing parallel region.

In [ ]:
%%cpp_omp -o code/additional/sections.cpp

#pragma omp parallel num_threads(2)
{
    #pragma omp sections
    {
        #pragma omp section
        {
            std::cout << "section 0, thread " << omp_get_thread_num() << std::endl;
        }

        #pragma omp section
        {
            std::cout << "section 1, thread " << omp_get_thread_num() << std::endl;
        }
    } //# implicit barrier without nowait clause
}

The parallel and sections constructs can also be fused into a single
```cpp
#pragma omp parallel sections
{
    // ...
}
```

## Flush


### The Problem

When synchronizing threads explicitly, or exchanging data between them, problems can arise.
Consider the following example which might hang indefinitely.

```cpp
int flag = 0;

#pragma omp parallel num_threads(2)
{
  if (omp_get_thread_num() == 0) {
    flag = 1;
  } else {
    while (!flag)
      std::this_thread::sleep_for(std::chrono::milliseconds(10));
  }
  /* some work */
}
```

The problem arises from the fact that writes from one thread to a shared variable might not be visible to other threads immediately.

### The Solution

Manually adding `flush` directives secures that changes are written to and read from memory ([OpenMP 5.1 - 2.19.8](https://www.openmp.org/spec-html/5.1/openmpsu106.html)).

<div class="alert alert-block alert-info"> <b>Note:</b> All (implicit and explicit) barriers imply flushing. </div>

An updated code version is below.
While it works, it is expected to be inefficient.
Better ways are using [atomic](synchronization.ipynb#Atomic), [masked](synchronization.ipynb#Masked-(replacing-Master)) or refactoring the code to split up the region.

In [ ]:
%%cpp_omp -o code/additional/flush.cpp

int flag = 0;

#pragma omp parallel num_threads(2)
{
  if (omp_get_thread_num() == 0) {
    flag = 1;
    #pragma omp flush
  } else {
    while (!flag) {
      #pragma omp flush
      std::this_thread::sleep_for(std::chrono::milliseconds(10));
    }
    /* some work */
  }
}